In [1]:

from pyspark.sql.functions import when,col,udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg

StatementMeta(, 33502ecf-0037-40a1-ab67-67f1fbcebf68, 5, Finished, Available)

In [5]:
df=spark.read.table("earthquake_events_silver")#.filter(col('time') > start_date)
display(df)
#to avoid the duplication of the start date we need to apply this filter

StatementMeta(, 33502ecf-0037-40a1-ab67-67f1fbcebf68, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3b59d1f7-f2c8-4b1c-8b90-4873bd93a42d)

In [10]:
def get_country_code(lat,lon):
    """

    retrieving the country code for the given latitude and longitude

    parameters:
    lat(float or str)
    lon(folat or str)

    returns:
    str:country code of the location,retrieved using the reverse geocoding API


    """
    coordinates=(float(lat),float(lon))
    return rg.search(coordinates)[0].get('cc')


#registering the udfs so they can be used on spark dataframes

get_country_code_udf=udf(get_country_code,StringType())


#adding the country code and city attributes
df_with_location=df.\
                withColumn("country_code",get_country_code_udf(col("latitude"),col("longitude")))

#adding the significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 33502ecf-0037-40a1-ab67-67f1fbcebf68, 14, Finished, Available)